In [1]:
import pandas as pd

import requests 

import os

import json

import googleapiclient.discovery
import googleapiclient.errors

import numpy as np

In [2]:
chiave_yt = json.load(open('key.json', 'r'))['youtube']['key'][1]
youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=chiave_yt)

In [3]:
#insert initial quota value
#MODIFICA MAXQUOTA A 9900!!
quota = 0

In [4]:
df_videos = pd.read_csv('data/get_youtuber_videos_output/youtuber_videos.csv')

In [5]:
def get_infos(response):
    info_comment_list=[]
    video_id=next_page_token = thread_comment_id = top_level_comment_id = author_display_name = author_channel_id = author_channel_id = text_original=None
    try:
        next_page_token = response['nextPageToken']
    except KeyError as e:
        #print(response)
        next_page_token = None
        print('Page token non esistente')
    items = response['items']
    for item in items:
        thread_comment_id = item['id']
        try:
            video_id = item['snippet']['videoId']
            top_level_comment = item['snippet']['topLevelComment']
            top_level_comment_id = top_level_comment['id']
            top_level_comment_snippet = top_level_comment['snippet']
            author_display_name = top_level_comment_snippet['authorDisplayName']
            text_original = top_level_comment_snippet['textOriginal']
            author_channel_id = top_level_comment_snippet['authorChannelId']['value']
        except KeyError as e:
            print(e.args[0])
            print('Problem with commentThread {}'.format(thread_comment_id))
        info_comment_list.append([video_id,thread_comment_id,top_level_comment_id,author_display_name,author_channel_id,text_original])
    return next_page_token, info_comment_list

In [ ]:
#############################################################################

In [6]:
#round 1
#la prossima volta togli e decommenta quello sotto

In [8]:
last_csv = int(np.load('data/get_comment_threads_output/last_csv.npy'))
next_page_token = str(np.load('data/get_comment_threads_output/next_page_token.npy'))
last_video_idx = int(np.load('data/get_comment_threads_output/video_to_start.npy'))
videos_not_finished = (np.load('data/get_comment_threads_output/videos_not_finished.npy')).tolist()
videos_disabled_comments = (np.load('data/get_comment_threads_output/videos_disabled_comments.npy')).tolist()
print("Starting with video #{}, next_page_token: {}".format(last_video_idx,next_page_token))
print("Writing on {} csv".format(last_csv+1))

Starting with video #781, next_page_token: QURTSl9pMnJqMWdmS2d2bndjN0tMODd2ZkJKS1pCbm9sX25fVy0zeV84X2dtUTRnMXJmdk5fQmgyX1FMb05IUEtWTXRaN25NWGF6RjF5SzZvVk1JanNVc2hvZEZ5UTU3X0Y5b19jS1NYLVp0SmdPbDhBMkd4UnVwWW43U3dGMF85Uk9fZUt6OUlsbExTS28=
Writing on 2 csv


In [9]:
df_videos_0 = df_videos[last_video_idx:]

In [10]:
#per i video con più di 10000 commenti, salvo l'indice del video con il next page token

In [11]:
comments = []
comments_count = 0

In [12]:
df_videos_0

,channel_id,video_id
781,UC3EFKdXAU99j3ppGgvTz7XQ,REWS3npz8e8
782,UC3EFKdXAU99j3ppGgvTz7XQ,Wyw1lYSoO0c
783,UC3EFKdXAU99j3ppGgvTz7XQ,9LSQc4eRKuA
784,UC3EFKdXAU99j3ppGgvTz7XQ,BkxXH1cl8c4
785,UC3EFKdXAU99j3ppGgvTz7XQ,elOE7SPf0vY
...,...,...
2588,UCq7EY7H2XF6TV7Z5mLv-aNg,bm-3Qrr7EUA
2589,UCq7EY7H2XF6TV7Z5mLv-aNg,hkxRjDS_Cjw
2590,UCq7EY7H2XF6TV7Z5mLv-aNg,8kElrRAa1ws
2591,UCq7EY7H2XF6TV7Z5mLv-aNg,7x6tX5T7LOs


In [14]:
for video in df_videos_0['video_id'].values:
    print('________________________________________________________')
    print('{} video: {}'.format(last_video_idx, video))
    video_comments_count = 0
    while next_page_token != None and video_comments_count<10000:
        print('Reached quotas: {}'.format(quota))
        if quota > 9900:
            print('Quota exceeded')
            break
        else:
            request = youtube.commentThreads().list(
                part="snippet",
                maxResults=100,
                pageToken=next_page_token,
                videoId=video
            )
            try:
                response = request.execute()
                quota += 3
                next_page_token, results = get_infos(response)
                video_comments_count+=len(results)
                comments_count+=video_comments_count
                comments.append(results)
            except:
                videos_disabled_comments.append(video)
                next_page_token = None
                print('Video has disabled comments')
    if quota > 9900:
        break
    if next_page_token == None or video_comments_count>=10000:
        if next_page_token != None:
            print('Reached max comment count')
            videos_not_finished.append([video, next_page_token])
        last_video_idx += 1
        next_page_token = ' '
        print('Total comments for {}: {}'.format(video, video_comments_count))

print("Done with {} comments, video to start: {}\n with next_page_token: {}".format(comments_count,last_video_idx, next_page_token))

________________________________________________________
781 video: REWS3npz8e8
Reached quotas: 0
Reached quotas: 3
Reached quotas: 6
Reached quotas: 9
Reached quotas: 12
Reached quotas: 15
Reached quotas: 18
Reached quotas: 21
Reached quotas: 24
Reached quotas: 27
Reached quotas: 30
Reached quotas: 33
Reached quotas: 36
Reached quotas: 39
Reached quotas: 42
Reached quotas: 45
Reached quotas: 48
Reached quotas: 51
Reached quotas: 54
Page token non esistente
Total comments for REWS3npz8e8: 1864
________________________________________________________
782 video: Wyw1lYSoO0c
Reached quotas: 57
Reached quotas: 60
Reached quotas: 63
Reached quotas: 66
Reached quotas: 69
Reached quotas: 72
Reached quotas: 75
Reached quotas: 78
Reached quotas: 81
Reached quotas: 84
Reached quotas: 87
Reached quotas: 90
Reached quotas: 93
Reached quotas: 96
Reached quotas: 99
Reached quotas: 102
Reached quotas: 105
Reached quotas: 108
Reached quotas: 111
Reached quotas: 114
Reached quotas: 117
Reached quotas: 

Reached quotas: 1092
Reached quotas: 1095
Reached quotas: 1098
Reached quotas: 1101
Reached quotas: 1104
Reached quotas: 1107
Reached quotas: 1110
Reached max comment count
Total comments for 8FchtMnGI0U: 10099
________________________________________________________
788 video: 5wphhWong3g
Reached quotas: 1113
Reached quotas: 1116
Reached quotas: 1119
Reached quotas: 1122
Reached quotas: 1125
Reached quotas: 1128
Reached quotas: 1131
Reached quotas: 1134
Reached quotas: 1137
Reached quotas: 1140
Reached quotas: 1143
Reached quotas: 1146
Reached quotas: 1149
Reached quotas: 1152
Reached quotas: 1155
Reached quotas: 1158
Reached quotas: 1161
Reached quotas: 1164
Reached quotas: 1167
Reached quotas: 1170
Reached quotas: 1173
Reached quotas: 1176
Reached quotas: 1179
Reached quotas: 1182
Reached quotas: 1185
Reached quotas: 1188
Reached quotas: 1191
Reached quotas: 1194
Reached quotas: 1197
Reached quotas: 1200
Reached quotas: 1203
Reached quotas: 1206
Reached quotas: 1209
Reached quotas: 

Reached quotas: 2181
Reached quotas: 2184
Reached quotas: 2187
Reached quotas: 2190
Reached quotas: 2193
Reached quotas: 2196
Reached quotas: 2199
Reached quotas: 2202
Reached quotas: 2205
Reached quotas: 2208
Reached quotas: 2211
Reached quotas: 2214
Reached quotas: 2217
Reached quotas: 2220
Reached quotas: 2223
Reached quotas: 2226
Reached quotas: 2229
Reached quotas: 2232
Reached quotas: 2235
Reached quotas: 2238
Reached quotas: 2241
Reached quotas: 2244
Reached quotas: 2247
Reached quotas: 2250
Reached quotas: 2253
Reached quotas: 2256
Reached quotas: 2259
Reached quotas: 2262
Reached quotas: 2265
Reached quotas: 2268
Reached quotas: 2271
Reached quotas: 2274
Reached quotas: 2277
Reached quotas: 2280
Reached quotas: 2283
Reached quotas: 2286
Reached quotas: 2289
Reached quotas: 2292
Reached quotas: 2295
Reached quotas: 2298
Reached quotas: 2301
Reached max comment count
Total comments for pjzJwhtE7QE: 10000
________________________________________________________
792 video: GzM_QUT

Reached quotas: 3231
Reached quotas: 3234
Reached quotas: 3237
Reached quotas: 3240
Reached quotas: 3243
Reached quotas: 3246
Reached quotas: 3249
Reached quotas: 3252
Reached quotas: 3255
Reached quotas: 3258
Reached quotas: 3261
Reached quotas: 3264
Reached quotas: 3267
Reached quotas: 3270
Reached quotas: 3273
Reached quotas: 3276
Reached quotas: 3279
Reached quotas: 3282
Reached quotas: 3285
Reached quotas: 3288
Reached quotas: 3291
Reached quotas: 3294
Reached quotas: 3297
Reached quotas: 3300
Reached quotas: 3303
Reached quotas: 3306
Reached quotas: 3309
Reached quotas: 3312
Reached quotas: 3315
Reached quotas: 3318
Reached quotas: 3321
Reached quotas: 3324
Reached quotas: 3327
Reached quotas: 3330
Reached quotas: 3333
Reached quotas: 3336
Reached quotas: 3339
Reached quotas: 3342
Reached quotas: 3345
Reached quotas: 3348
Reached quotas: 3351
Reached quotas: 3354
Reached quotas: 3357
Reached quotas: 3360
Reached quotas: 3363
Reached quotas: 3366
Reached quotas: 3369
Reached quota

Reached quotas: 4320
Reached quotas: 4323
Reached quotas: 4326
Reached quotas: 4329
Reached quotas: 4332
Reached quotas: 4335
Reached quotas: 4338
Reached quotas: 4341
Reached quotas: 4344
Reached quotas: 4347
Reached quotas: 4350
Reached quotas: 4353
Reached quotas: 4356
Reached quotas: 4359
Reached quotas: 4362
Reached quotas: 4365
Reached quotas: 4368
Reached quotas: 4371
Reached quotas: 4374
Reached quotas: 4377
Reached quotas: 4380
Reached quotas: 4383
Reached quotas: 4386
Reached quotas: 4389
Reached quotas: 4392
Reached quotas: 4395
Reached quotas: 4398
Reached quotas: 4401
Reached quotas: 4404
Reached quotas: 4407
Reached quotas: 4410
Reached quotas: 4413
Reached quotas: 4416
Reached quotas: 4419
Reached quotas: 4422
Reached quotas: 4425
Reached quotas: 4428
Reached quotas: 4431
Reached quotas: 4434
Reached quotas: 4437
Reached quotas: 4440
Reached quotas: 4443
Reached quotas: 4446
Reached quotas: 4449
Reached quotas: 4452
Reached quotas: 4455
Reached quotas: 4458
Reached quota

...

Reached quotas: 7353
Reached quotas: 7356
Reached quotas: 7359
Reached quotas: 7362
Reached quotas: 7365
Reached quotas: 7368
Reached quotas: 7371
Reached quotas: 7374
Page token non esistente
Total comments for ZEkIpNPdt2w: 2548
________________________________________________________
872 video: x9eyvSa-WJk
Reached quotas: 7377
Reached quotas: 7380
Reached quotas: 7383
Reached quotas: 7386
Reached quotas: 7389
Reached quotas: 7392
Reached quotas: 7395
Reached quotas: 7398
Reached quotas: 7401
Reached quotas: 7404
Reached quotas: 7407
Reached quotas: 7410
Reached quotas: 7413
Reached quotas: 7416
Reached quotas: 7419
Reached quotas: 7422
Reached quotas: 7425
Reached quotas: 7428
Reached quotas: 7431
Reached quotas: 7434
Reached quotas: 7437
Reached quotas: 7440
Reached quotas: 7443
Reached quotas: 7446
Reached quotas: 7449
Reached quotas: 7452
Reached quotas: 7455
Reached quotas: 7458
Reached quotas: 7461
Reached quotas: 7464
Reached quotas: 7467
Reached quotas: 7470
Reached quotas: 74

Reached quotas: 7962
Reached quotas: 7965
Reached quotas: 7968
Reached quotas: 7971
Reached quotas: 7974
Reached quotas: 7977
Reached quotas: 7980
Reached quotas: 7983
Reached quotas: 7986
Reached quotas: 7989
Page token non esistente
Total comments for eX4bhGpS410: 2047
________________________________________________________
900 video: bGrqKydQ1qE
Reached quotas: 7992
Reached quotas: 7995
Reached quotas: 7998
Reached quotas: 8001
Reached quotas: 8004
Page token non esistente
Total comments for bGrqKydQ1qE: 444
________________________________________________________
901 video: 43pj8UVbfDs
Reached quotas: 8007
Reached quotas: 8010
Reached quotas: 8013
Reached quotas: 8016
Reached quotas: 8019
Reached quotas: 8022
Reached quotas: 8025
Reached quotas: 8028
Reached quotas: 8031
Page token non esistente
Total comments for 43pj8UVbfDs: 894
________________________________________________________
902 video: sJGkUVA4UwE
Reached quotas: 8034
Reached quotas: 8037
Reached quotas: 8040
Reached q

Reached quotas: 8619
Reached quotas: 8622
Reached quotas: 8625
Reached quotas: 8628
Reached quotas: 8631
Reached quotas: 8634
Reached quotas: 8637
Reached quotas: 8640
Reached quotas: 8643
Reached quotas: 8646
Reached quotas: 8649
Reached quotas: 8652
Reached quotas: 8655
Page token non esistente
Total comments for xyZ2hVe_mbU: 1404
________________________________________________________
925 video: MJH8JcZvCxM
Reached quotas: 8658
Reached quotas: 8661
Reached quotas: 8664
Reached quotas: 8667
Reached quotas: 8670
Reached quotas: 8673
Reached quotas: 8676
Reached quotas: 8679
Reached quotas: 8682
Reached quotas: 8685
Reached quotas: 8688
Reached quotas: 8691
Reached quotas: 8694
Reached quotas: 8697
Reached quotas: 8700
Reached quotas: 8703
Reached quotas: 8706
Reached quotas: 8709
Reached quotas: 8712
Reached quotas: 8715
Page token non esistente
Total comments for MJH8JcZvCxM: 1962
________________________________________________________
926 video: Lfo1aHtoAZo
Reached quotas: 8718
Re

Page token non esistente
Total comments for 5fErydKT4X0: 289
________________________________________________________
950 video: -oMg14BZg6o
Reached quotas: 9282
Reached quotas: 9285
Reached quotas: 9288
Page token non esistente
Total comments for -oMg14BZg6o: 202
________________________________________________________
951 video: 4GkMMlsIur8
Reached quotas: 9291
Reached quotas: 9294
Reached quotas: 9297
Reached quotas: 9300
Page token non esistente
Total comments for 4GkMMlsIur8: 392
________________________________________________________
952 video: 0f77Gg8EfPI
Reached quotas: 9303
Reached quotas: 9306
Reached quotas: 9309
Reached quotas: 9312
Reached quotas: 9315
Reached quotas: 9318
Page token non esistente
Total comments for 0f77Gg8EfPI: 550
________________________________________________________
953 video: VUsQg_kPQfs
Reached quotas: 9321
Page token non esistente
Total comments for VUsQg_kPQfs: 70
________________________________________________________
954 video: OqoKo_grtJo
Re

Reached quotas: 9696
Reached quotas: 9699
Reached quotas: 9702
Page token non esistente
Total comments for IQE5jI1dilI: 388
________________________________________________________
988 video: -wwFs7dhkpg
Reached quotas: 9705
Reached quotas: 9708
Reached quotas: 9711
Reached quotas: 9714
Page token non esistente
Total comments for -wwFs7dhkpg: 364
________________________________________________________
989 video: YllNOc2oy5o
Reached quotas: 9717
Reached quotas: 9720
Reached quotas: 9723
Reached quotas: 9726
Reached quotas: 9729
Page token non esistente
Total comments for YllNOc2oy5o: 461
________________________________________________________
990 video: ZqbzJinqamM
Reached quotas: 9732
Reached quotas: 9735
Reached quotas: 9738
Reached quotas: 9741
Reached quotas: 9744
Reached quotas: 9747
Reached quotas: 9750
Reached quotas: 9753
Reached quotas: 9756
Page token non esistente
Total comments for ZqbzJinqamM: 862
________________________________________________________
991 video: y9ywkC1

In [15]:
np.save('data/get_comment_threads_output/next_page_token.npy',next_page_token)
np.save('data/get_comment_threads_output/video_to_start.npy',last_video_idx)
np.save('data/get_comment_threads_output/videos_not_finished.npy',videos_not_finished)
np.save('data/get_comment_threads_output/videos_disabled_comments.npy',videos_disabled_comments)

In [16]:
print(next_page_token, last_video_idx)

QURTSl9pM211d1UzelB0bXhkZnY2Wm9DdXo2LU9jZE56QmRKN3Fxa1V2R2VzWWpnYlVlaEVyNzFWa3RwWGg0aFBseUlCQXR4aEozZmVDcldpc1MwUkIydzlQNjZZaDludno2emU5QTJGeXRlVlpYV21FWkZwdDBtLUlKUjMxV3pwQVVEakJlaDlRZXBnbDQ= 999


In [17]:
format_ok_comments = []
for outer_comment in comments:
    for comment in outer_comment:
        format_ok_comments.append(comment)

In [18]:
format_ok_comments

[['REWS3npz8e8',
  'UgyUkApnjoPW5VaEJgp4AaABAg',
  'UgyUkApnjoPW5VaEJgp4AaABAg',
  'Ms Jinan',
  'UCiU62SP7Z7ypFVzHjvvzjTQ',
  'Ugh. Now I want avocados with milk, ice, and condensed milk 🤤'],
 ['REWS3npz8e8',
  'Ugw5LxHg9wQTtfAQndJ4AaABAg',
  'Ugw5LxHg9wQTtfAQndJ4AaABAg',
  'Gabriela Lorraine',
  'UCjGrBDSuIkrSA2oxJ2SGxrA',
  'omg mae looks just like her mom 😍'],
 ['REWS3npz8e8',
  'Ugyc-aELIU4fBPPSLEV4AaABAg',
  'Ugyc-aELIU4fBPPSLEV4AaABAg',
  'TiNkLe WiNklE',
  'UCFd_h453mrtA4m8byx_3Tug',
  "Dude I read that as *we're black and we're eating our sister's pregnancy test*"],
 ['REWS3npz8e8',
  'UgzRddzTwSgadBjX5Vt4AaABAg',
  'UgzRddzTwSgadBjX5Vt4AaABAg',
  'Kimaya Henry',
  'UC95ikGQpgHRsuacx5lyDy3w',
  'FINALLYYYYY!!love you guyss❤️'],
 ['REWS3npz8e8',
  'UgyRIsRgzCAQgO4rxBJ4AaABAg',
  'UgyRIsRgzCAQgO4rxBJ4AaABAg',
  'Mitchell',
  'UCrDZjkEKpTehFHGuSRIP0ZA',
  "We're BACK and We're eating my sister's Pregnancy"],
 ['REWS3npz8e8',
  'UgzLzJq1TjpvtxloXiV4AaABAg',
  'UgzLzJq1TjpvtxloXiV4

In [19]:
df_comment_thread = pd.DataFrame(format_ok_comments,columns=['video_id','comment_thread_id','top_level_comment_id','author_display_name','author_channel_id','text_original'])

In [20]:
df_comment_thread

,video_id,comment_thread_id,top_level_comment_id,author_display_name,author_channel_id,text_original
0,REWS3npz8e8,UgyUkApnjoPW5VaEJgp4AaABAg,UgyUkApnjoPW5VaEJgp4AaABAg,Ms Jinan,UCiU62SP7Z7ypFVzHjvvzjTQ,"Ugh. Now I want avocados with milk, ice, and c..."
1,REWS3npz8e8,Ugw5LxHg9wQTtfAQndJ4AaABAg,Ugw5LxHg9wQTtfAQndJ4AaABAg,Gabriela Lorraine,UCjGrBDSuIkrSA2oxJ2SGxrA,omg mae looks just like her mom 😍
2,REWS3npz8e8,Ugyc-aELIU4fBPPSLEV4AaABAg,Ugyc-aELIU4fBPPSLEV4AaABAg,TiNkLe WiNklE,UCFd_h453mrtA4m8byx_3Tug,Dude I read that as *we're black and we're eat...
3,REWS3npz8e8,UgzRddzTwSgadBjX5Vt4AaABAg,UgzRddzTwSgadBjX5Vt4AaABAg,Kimaya Henry,UC95ikGQpgHRsuacx5lyDy3w,FINALLYYYYY!!love you guyss❤️
4,REWS3npz8e8,UgyRIsRgzCAQgO4rxBJ4AaABAg,UgyRIsRgzCAQgO4rxBJ4AaABAg,Mitchell,UCrDZjkEKpTehFHGuSRIP0ZA,We're BACK and We're eating my sister's Pregnancy
...,...,...,...,...,...,...
320804,puxXSTyYh7s,UgwgG1ENilUkW5jjoIV4AaABAg,UgwgG1ENilUkW5jjoIV4AaABAg,Mima Vojtičková,UCs1iEf-qXkeyVv_kzYm7C1w,"Why do beauty gurus name their videos ""everyda..."
320805,puxXSTyYh7s,UgxBPU7v0dQNlO-CmCZ4AaABAg,UgxBPU7v0dQNlO-CmCZ4AaABAg,ToxicTerrorist,UCq653A1tI4ZhAqErBWlWS8g,Your hot
320806,puxXSTyYh7s,UgwgLhS5hADK0Tr1OvJ4AaABAg,UgwgLhS5hADK0Tr1OvJ4AaABAg,luna,UCTtCfn8mNBIJGy0_UL9Qq4Q,Before she put on the tinted moisturizer her h...
320807,puxXSTyYh7s,UgzCt3vnSD86zx91vS54AaABAg,UgzCt3vnSD86zx91vS54AaABAg,Stefan Turan,UCnZxoTJ9Md_UoEmhVZQNABg,Mommy and Daddy are selling the house for more...


In [21]:
last_csv

1

In [22]:
last_csv+=1
last_csv

2

In [23]:
df_comment_thread.to_csv('data/get_comment_threads_output/df_comment_thread_{}.csv'.format(last_csv), index = False)

In [24]:
np.save('data/get_comment_threads_output/last_csv.npy',last_csv)

In [25]:
df = pd.read_csv('data/get_comment_threads_output/df_comment_thread_{}.csv'.format(last_csv))
df

,video_id,comment_thread_id,top_level_comment_id,author_display_name,author_channel_id,text_original
0,REWS3npz8e8,UgyUkApnjoPW5VaEJgp4AaABAg,UgyUkApnjoPW5VaEJgp4AaABAg,Ms Jinan,UCiU62SP7Z7ypFVzHjvvzjTQ,"Ugh. Now I want avocados with milk, ice, and c..."
1,REWS3npz8e8,Ugw5LxHg9wQTtfAQndJ4AaABAg,Ugw5LxHg9wQTtfAQndJ4AaABAg,Gabriela Lorraine,UCjGrBDSuIkrSA2oxJ2SGxrA,omg mae looks just like her mom 😍
2,REWS3npz8e8,Ugyc-aELIU4fBPPSLEV4AaABAg,Ugyc-aELIU4fBPPSLEV4AaABAg,TiNkLe WiNklE,UCFd_h453mrtA4m8byx_3Tug,Dude I read that as *we're black and we're eat...
3,REWS3npz8e8,UgzRddzTwSgadBjX5Vt4AaABAg,UgzRddzTwSgadBjX5Vt4AaABAg,Kimaya Henry,UC95ikGQpgHRsuacx5lyDy3w,FINALLYYYYY!!love you guyss❤️
4,REWS3npz8e8,UgyRIsRgzCAQgO4rxBJ4AaABAg,UgyRIsRgzCAQgO4rxBJ4AaABAg,Mitchell,UCrDZjkEKpTehFHGuSRIP0ZA,We're BACK and We're eating my sister's Pregnancy
...,...,...,...,...,...,...
320804,puxXSTyYh7s,UgwgG1ENilUkW5jjoIV4AaABAg,UgwgG1ENilUkW5jjoIV4AaABAg,Mima Vojtičková,UCs1iEf-qXkeyVv_kzYm7C1w,"Why do beauty gurus name their videos ""everyda..."
320805,puxXSTyYh7s,UgxBPU7v0dQNlO-CmCZ4AaABAg,UgxBPU7v0dQNlO-CmCZ4AaABAg,ToxicTerrorist,UCq653A1tI4ZhAqErBWlWS8g,Your hot
320806,puxXSTyYh7s,UgwgLhS5hADK0Tr1OvJ4AaABAg,UgwgLhS5hADK0Tr1OvJ4AaABAg,luna,UCTtCfn8mNBIJGy0_UL9Qq4Q,Before she put on the tinted moisturizer her h...
320807,puxXSTyYh7s,UgzCt3vnSD86zx91vS54AaABAg,UgzCt3vnSD86zx91vS54AaABAg,Stefan Turan,UCnZxoTJ9Md_UoEmhVZQNABg,Mommy and Daddy are selling the house for more...
